In [126]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

from funcs import *
from config import *
from plotfuncs import *

import warnings
warnings.filterwarnings('ignore')

In [154]:
init_req = '2019080709'
init_time = get_init(init_req)
pdata = load_plumedata(init_time, 'KSLC')
pdata

Loading plume data for KSLC from file


<xarray.Dataset>
Dimensions:    (member: 26, time: 30)
Coordinates:
  * time       (time) float64 1.565e+18 1.565e+18 ... 1.565e+18 1.565e+18
    lat        float32 40.77083
    lon        float32 -111.97084
  * member     (member) float64 1.0 2.0 3.0 4.0 5.0 ... 22.0 23.0 24.0 25.0 26.0
    member_id  (member) <U7 'arw_ctl' 'arw_n1' 'arw_n2' ... 'nmb_p5' 'nmb_p6'
Data variables:
    slr        (member, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    dqpf       (member, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    snow       (member, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    acc_qpf    (member, time) float32 0.0 0.0 0.0 ... 0.035470035 0.035470035
    acc_snow   (member, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes:
    description:  Downscaled {} QPF/Snow Grids Init SREF UTC
    history:      Created 2019-08-07 13:23:04.301100
    source:       University of Utah - Steenburgh Research Group

In [155]:
time = pd.to_datetime(pdata.time.values)
timefmt = '%Y-%m-%d %H:%M'
p1i = np.where(np.array([t.hour for t in time]) == 18)[0][0]

In [172]:
pop, pos = [], []
for i, t in enumerate(time):
    n = pdata.isel(time=i).dqpf.size

    p_yes = np.where(pdata.isel(time=i).dqpf >= 0.01)[0].size
    pop.append(np.round(p_yes/n*100, 0).astype(int))

    s_yes = np.where(pdata.isel(time=i).snow >= 0.1)[0].size
    pos.append(np.round(s_yes/n*100, 0).astype(int))

pop, pos = np.array(pop), np.array(pos)
pqpf = pdata.dqpf.quantile([.1, .5, .9], dim='member')
pqsf = pdata.snow.quantile([.1, .5, .9], dim='member')

In [173]:
# Use np.prod(times*members)
p1 = pdata.isel(time=slice(p1i+1, p1i+3))
n1 = np.prod(p1.dqpf.shape)

p1_pop = int(np.round((np.where(p1.dqpf >= 0.01)[0].size/n1)*100, 0))
p1_pos = int(np.round((np.where(p1.snow >= 0.01)[0].size/n1)*100, 0))
p1_pop, p1_pos

(0, 0)

In [174]:
p2 = pdata.isel(time=slice(p1i+4, p1i+11))
n2 = np.prod(p2.dqpf.shape)

p2_pop = int(np.round((np.where(p2.dqpf >= 0.01)[0].size/n2)*100, 0))
p2_pos = int(np.round((np.where(p2.snow >= 0.01)[0].size/n2)*100, 0))
p2_pop, p2_pos

(9, 0)

In [186]:
p1_pqpf = [np.round(v, 2) for v in p1.dqpf.sum(dim='time').quantile([.1, .5, .9], dim='member').values]
p1_pqsf = [np.round(v, 2) for v in p1.snow.sum(dim='time').quantile([.1, .5, .9], dim='member').values]
p1_pqpf, p1_pqsf

([0.0, 0.0, 0.0], [0.0, 0.0, 0.0])

In [187]:
p2_pqpf = p2.dqpf.sum(dim='time').quantile([.1, .5, .9], dim='member').values
p2_pqsf = p2.snow.sum(dim='time').quantile([.1, .5, .9], dim='member').values
p2_pqpf, p2_pqsf

(array([0.        , 0.00400052, 0.0698779 ]), array([0., 0., 0.]))

In [198]:
try:
    with open(wd + 'forecast.csv', 'r+') as rfp:
        lines = rfp.readlines()
        lenlines = len(lines)
except:
    lines = []
    lenlines = 0

maxlines = 91 if lenlines > 91 else None

with open(wd + 'forecast.csv', 'w+') as wfp:
    wfp.write(' , , , , \n')
    wfp.write('{}, {}, {}, {}, {}\n'.format(
        'KSLC Downscaled SREF Init: '+init_time.strftime('%Y-%m-%d %H%M UTC'), 
        'PoP (%)', 
        'PQPF 10/50/90 (in)', 
        'PoS (%)', 
        'PQSF 10/50/90 (in)'))

    for i in range(11): 
        wfp.write('{}, {}, {}, {}, {}\n'.format(
            '3-h Period ending: ' + time[p1i+i].strftime(timefmt),
            pop[p1i+i],
            str(['{:6.3f}'.format(round(v[0], 3)) for v in pqpf.isel(time=[p1i+i]).values]).replace(',', ' ').replace("'",'').replace('[', '').replace(']', ''),
            pos[p1i+i],
            str(['{:6.3f}'.format(round(v[0], 3)) for v in pqsf.isel(time=[p1i+i]).values]).replace(',', ' ').replace("'",'').replace('[', '').replace(']', '')))
        
        if i in [2, 10]:
            period = 'Period 1 (%s – %s)'%(time[p1i], time[p1i+i]) if i == 2 else 'Period 2 (%s – %s)'%(time[p1i+10], time[p1i+i])
            _pop, _pos = (p1_pop, p1_pos) if i == 2 else (p2_pop, p2_pos)
            _pqpf, _pqsf = (p1_pqpf, p1_pqsf) if i == 2 else (p2_pqpf, p2_pqsf)
            
            wfp.write('\n{}, {}, {}, {}, {}\n\n'.format(
                period,
                _pop,
                str(['{:6.3f}'.format(round(v, 3)) for v in _pqpf]).replace(',', ' ').replace("'",'').replace('[', '').replace(']', ''),
                _pos,
                str(['{:6.3f}'.format(round(v, 3)) for v in _pqsf]).replace(',', ' ').replace("'",'').replace('[', '').replace(']', '')
            ))

#     [wfp.write(line) for line in lines[:maxlines]]

In [193]:
time[p1i+10]

Timestamp('2019-08-09 00:00:00')